In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/all-prompts/all_prompts.csv
/kaggle/input/train-shl/train (1).csv
/kaggle/input/test-shl/test.csv


preprocess
word to vec
lstm
train
predict

In [2]:
#pd.set_option("display.max_colwidth", None)

In [3]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score

In [4]:
d = pd.read_csv("/kaggle/input/train-shl/train (1).csv")
d.head()

,Unnamed: 0,promptId,uniqueId,essay,evaluator_rating
0,0,1,1_323,"At present age, our education system is not go...",3.0
1,1,1,1_238,I am agree the tightly defined curriculum of o...,4.0
2,2,1,1_212,I strongly agree with the statement that tight...,2.0
3,3,1,1_117,Our education system is nice quitely but i dis...,2.0
4,4,1,1_229,i am totally agree with the statement that tig...,3.0


In [5]:
from sklearn.utils import shuffle
d = shuffle(d)

In [6]:
d['essay'] = d['essay'].fillna('').apply(str)

In [7]:
d.isnull().sum()

Unnamed: 0          0
promptId            0
uniqueId            0
essay               0
evaluator_rating    0
dtype: int64

In [8]:
d[d["evaluator_rating"] == 0]

,Unnamed: 0,promptId,uniqueId,essay,evaluator_rating
440,440,2,2_217,RICH &amp; POOR Today in our country peoples m...,0.0
87,87,1,1_338,Accordingly Iwant to say this to the tightly o...,0.0
973,973,5,5_192,By using the nuclear powers the job is becomin...,0.0
1176,1176,5,5_279,"on the high seas,the wrapping of wooden ships ...",0.0
144,144,1,1_305,curriculum of our education systemour educatio...,0.0
...,...,...,...,...,...
969,969,5,5_95,human being is most impornt in today and produ...,0.0
986,986,5,5_118,The arms and weapens that are used in wars may...,0.0
1183,1183,5,5_228,NUCLEAR DEVELOPMENT:The era of nuclear develop...,0.0
1180,1180,5,5_427,"IT is my concerend this is not true ,the crimi...",0.0


In [9]:
"""promptid = d["promptId"]
pID_train = promptid[:1200]
pID_test = promptid[1200:]"""

'promptid = d["promptId"]\npID_train = promptid[:1200]\npID_test = promptid[1200:]'

In [10]:
"""d.drop(columns= ["Unnamed: 0","promptId", "uniqueId"],inplace = True)
d1 = d[ :1200]
d2 = d[1200:]"""


'd.drop(columns= ["Unnamed: 0","promptId", "uniqueId"],inplace = True)\nd1 = d[ :1200]\nd2 = d[1200:]'

In [11]:
#X = d[["essay","promptId"]]

In [12]:
d1 = d[:900]
d2 = d[900:]
x_train = d1.essay
x_test = d2.essay
prompt_train = d1.promptId
prompt_test = d2.promptId
y_train = d1.evaluator_rating
y_test = d2.evaluator_rating


In [13]:
#x_train, x_test, y_train, y_test = train_test_split(, d.evaluator_rating, test_size=0.2, random_state=42)

In [14]:
type(x_train)

pandas.core.series.Series

In [15]:
"""train_id = x_train.promptId
x_train.drop(columns = ["promptId"], axis = 1,inplace = True)
test_id = x_test.promptId
x_test.drop(columns = ["promptId"], axis = 1,inplace = True)
x_train"""

'train_id = x_train.promptId\nx_train.drop(columns = ["promptId"], axis = 1,inplace = True)\ntest_id = x_test.promptId\nx_test.drop(columns = ["promptId"], axis = 1,inplace = True)\nx_train'

In [16]:
y_train

440     0.0
792     4.0
827     2.5
1170    2.5
92      3.5
       ... 
241     2.5
74      3.0
688     3.5
202     3.0
1221    2.5
Name: evaluator_rating, Length: 900, dtype: float64

In [17]:
t = pd.read_csv("/kaggle/input/test-shl/test.csv")
t.head()

,Unnamed: 0,promptId,uniqueId,essay
0,0,1,1_315,Curriculum has been adopted in many schools. T...
1,1,1,1_214,"I strongly agree with the statement , The tig..."
2,2,1,1_196,Imagination and creativity is the most importa...
3,3,1,1_178,In our eduction system leaves no room for imag...
4,4,1,1_201,"I will agree at some what extend, because if w..."


In [18]:
p = pd.read_csv("/kaggle/input/all-prompts/all_prompts.csv")
print(p)

   Unnamed: 0  promptId                                    prompt_question
0           0         1  The tight curriculum of our education system l...
1           1         2  Our society is disrupted by the ever-widening ...
2           2         3  Has technology become a new addiction? Have we...
3           4         5  In the nuclear age, the production and develop...


In [19]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords   

In [20]:
x_train

440     RICH &amp; POOR Today in our country peoples m...
792     In this era of 21st century, we are surrounded...
827     Yes, I agree with this Statement (Has technolo...
1170    the race in the production of weapons in the e...
92      The Education System Of IndiaThe education sys...
                              ...                        
241     As we all know the the education system of ind...
74      Education is provided in order to make one und...
688     TECHNOLOGY: BOON OR BAN??Technology , a word t...
202     OUR EDUCATION SYSTEM-Eduction in india as prov...
1221    Nuclear development have both pros and cons ac...
Name: essay, Length: 900, dtype: object

In [21]:
"""x_train = list(x_train)
x_test = list(x_test)"""

'x_train = list(x_train)\nx_test = list(x_test)'

In [22]:
x_train

440     RICH &amp; POOR Today in our country peoples m...
792     In this era of 21st century, we are surrounded...
827     Yes, I agree with this Statement (Has technolo...
1170    the race in the production of weapons in the e...
92      The Education System Of IndiaThe education sys...
                              ...                        
241     As we all know the the education system of ind...
74      Education is provided in order to make one und...
688     TECHNOLOGY: BOON OR BAN??Technology , a word t...
202     OUR EDUCATION SYSTEM-Eduction in india as prov...
1221    Nuclear development have both pros and cons ac...
Name: essay, Length: 900, dtype: object

In [23]:
import re
stop_words = set(stopwords.words('english'))
def preprocess_sentence(w):
    #changing all character to lowercase
    w = w.lower().strip()
    #print(w)
    w = re.sub(r"([?.!,¿])", r" \1 ", (w))
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.rstrip().strip()
    return w

def sentword(w):
    sentence = [] 
    words=w.split()
    for i in words:
        if i not in stop_words: 
            sentence.append(i)
    return sentence
#x_train = preprocess_sentence(x_train)
x_train = x_train.apply(preprocess_sentence)
#d1["essay"] = d1["essay"].apply(sentword)
x_test = x_test.apply(preprocess_sentence)
#d2["essay"] = d2["essay"].apply(sentword)

In [24]:
"""x_train = d1.essay
x_test = d2.essay"""

'x_train = d1.essay\nx_test = d2.essay'

In [25]:
#x_train

In [26]:
"""x_train = list(x_train)
x_test = list(x_test)
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(binary = True)
tf = tfidf.fit(x_train)
x_train = tf.transform(x_train)
x_test = tf.transform(x_test)"""


'x_train = list(x_train)\nx_test = list(x_test)\nfrom sklearn.feature_extraction.text import TfidfVectorizer\ntfidf = TfidfVectorizer(binary = True)\ntf = tfidf.fit(x_train)\nx_train = tf.transform(x_train)\nx_test = tf.transform(x_test)'

In [27]:
#x_train.shape

In [28]:
"""from sklearn.decomposition import TruncatedSVD
clf = TruncatedSVD(300)
Xpca = clf.fit_transform(x_train)"""

'from sklearn.decomposition import TruncatedSVD\nclf = TruncatedSVD(300)\nXpca = clf.fit_transform(x_train)'

In [29]:
#Xpca_test = clf.transform(x_test)

In [30]:
#prompt_train

In [31]:
#np.append(Xpca,prompt_train)


In [32]:
#np.append(Xpca_test,prompt_test)

In [33]:
#Xpca_test.shape

In [34]:
#Xpca["promptId"] = d.promptId

In [35]:
#Xpca.shape

In [36]:
#Xpca = np.reshape(Xpca, (Xpca.shape[0], 1, Xpca.shape[1]))
#Xpca_test = np.reshape(Xpca_test, (Xpca_test.shape[0], 1, Xpca_test.shape[1]))

#testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))

In [37]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model
lstm = get_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 300)            721200    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________


In [38]:
#Xpca.shape

In [39]:
def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index_to_key)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model.wv[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [40]:
sentences = []
for essay in d1.essay:
# Obtaining all sentences from the training essays.
    sentences += essay_to_sentences(essay, remove_stopwords = True)

In [41]:
import gensim
from gensim.models import Word2Vec

num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

print("Training Word2Vec Model...")
model = Word2Vec(sentences, workers=num_workers, vector_size=num_features, min_count = min_word_count, window = context, sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

clean_train_essays = []
    
# Generate training and testing data word vectors.
for essay_v in d1.essay:
    clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
clean_test_essays = []
for essay_v in d2.essay:
    clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
trainDataVecs = np.array(trainDataVecs)
testDataVecs = np.array(testDataVecs)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
#trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
#testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    #lstm_model = get_model()
    #lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50

Training Word2Vec Model...


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  del sys.path[0]


In [42]:
trainDataVecs.shape

(900, 300)

In [43]:
np.append(trainDataVecs,prompt_train)

array([ 0.02817143,  0.09923932, -0.02168802, ...,  3.        ,
        1.        ,  5.        ])

In [44]:
np.append(testDataVecs,prompt_test)

array([ 0.02042537,  0.07195824, -0.05602362, ...,  1.        ,
        1.        ,  1.        ])

In [45]:
trainDataVecs.shape

(900, 300)

In [46]:
trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))

In [47]:
trainDataVecs.shape

(900, 1, 300)

In [48]:
lstm.fit(trainDataVecs, y_train, batch_size=64, epochs=150)

Epoch 1/150
15/15 [==============================] - 4s 15ms/step - loss: 6.3522 - mae: 2.2767
Epoch 2/150
15/15 [==============================] - 0s 13ms/step - loss: 1.2464 - mae: 0.8581
Epoch 3/150
15/15 [==============================] - 0s 13ms/step - loss: 1.2430 - mae: 0.8739
Epoch 4/150
15/15 [==============================] - 0s 13ms/step - loss: 1.1644 - mae: 0.8168
Epoch 5/150
15/15 [==============================] - 0s 13ms/step - loss: 1.2077 - mae: 0.8472
Epoch 6/150
15/15 [==============================] - 0s 13ms/step - loss: 1.0663 - mae: 0.7780
Epoch 7/150
15/15 [==============================] - 0s 14ms/step - loss: 1.2167 - mae: 0.8606
Epoch 8/150
15/15 [==============================] - 0s 13ms/step - loss: 1.3053 - mae: 0.8592
Epoch 9/150
15/15 [==============================] - 0s 14ms/step - loss: 1.2291 - mae: 0.8436
Epoch 10/150
15/15 [==============================] - 0s 14ms/step - loss: 1.2986 - mae: 0.8560
Epoch 11/150
15/15 [=============================

In [49]:
#validation_data = (testDataVecs,y_test),

In [50]:
y_pred = lstm.predict(testDataVecs)
y_pred = np.around(y_pred)
#y_pred

In [51]:
from sklearn.metrics import r2_score
acc = r2_score(y_test, y_pred)

In [52]:
acc

-0.07625627004501223

In [53]:
#y_pred

In [54]:
"""from sklearn.decomposition import PCA
pca = PCA(n_components=300)
x = pca.fit(x_train)"""

'from sklearn.decomposition import PCA\npca = PCA(n_components=300)\nx = pca.fit(x_train)'

In [55]:
""".toarray()
.toarray()
x_train = x_train[None:, :,]
x_test = x_test[None:, :, ]"""

'.toarray()\n.toarray()\nx_train = x_train[None:, :,]\nx_test = x_test[None:, :, ]'

In [56]:
"""from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
lr = LinearRegression().fit(x_train, d1.evaluator_rating)
y_pred = lr.predict(x_test)
mse = r2_score(d2.evaluator_rating, y_pred)
mse"""

'from sklearn.linear_model import LinearRegression\nfrom sklearn.metrics import mean_squared_error\nfrom sklearn.metrics import r2_score\nlr = LinearRegression().fit(x_train, d1.evaluator_rating)\ny_pred = lr.predict(x_test)\nmse = r2_score(d2.evaluator_rating, y_pred)\nmse'

In [57]:
"""from sklearn.linear_model import Ridge
rd = Ridge().fit(x_train, d1.evaluator_rating)
y_pred1 = rd.predict(x_test)
mse1 = r2_score(d2.evaluator_rating,y_pred1)
mse1"""

'from sklearn.linear_model import Ridge\nrd = Ridge().fit(x_train, d1.evaluator_rating)\ny_pred1 = rd.predict(x_test)\nmse1 = r2_score(d2.evaluator_rating,y_pred1)\nmse1'

In [58]:
"""import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional,BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.utils import  to_categorical"""

'import pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import LabelEncoder\nfrom keras.models import Model\nfrom keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional,BatchNormalization\nfrom keras.optimizers import RMSprop\nfrom keras.preprocessing.text import Tokenizer\nfrom keras.preprocessing import sequence\nfrom keras.utils import to_categorical\nfrom keras.callbacks import EarlyStopping\nfrom keras.utils import  to_categorical'

In [59]:
"""max_words = 2500
max_len = 500
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = Bidirectional(LSTM(64))(layer)
    #layer = Bidirectional(LSTM(128))(layer)
    layer = Dense(64,name='FC1')(layer)
    #layer=BatchNormalization()(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(64,name='FC2')(layer)
    #layer=BatchNormalization()(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(4,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model"""

"max_words = 2500\nmax_len = 500\ndef RNN():\n    inputs = Input(name='inputs',shape=[max_len])\n    layer = Embedding(max_words,50,input_length=max_len)(inputs)\n    layer = Bidirectional(LSTM(64))(layer)\n    #layer = Bidirectional(LSTM(128))(layer)\n    layer = Dense(64,name='FC1')(layer)\n    #layer=BatchNormalization()(layer)\n    layer = Activation('relu')(layer)\n    layer = Dropout(0.5)(layer)\n    layer = Dense(64,name='FC2')(layer)\n    #layer=BatchNormalization()(layer)\n    layer = Activation('relu')(layer)\n    layer = Dropout(0.5)(layer)\n    layer = Dense(4,name='out_layer')(layer)\n    layer = Activation('softmax')(layer)\n    model = Model(inputs=inputs,outputs=layer)\n    return model"

In [60]:
"""model = RNN()
print(model.summary())
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

model.fit(x_train, d1.evaluator_rating,batch_size=64,epochs=2,validation_data=(x_test,d2.evaluator_rating))"""

"model = RNN()\nprint(model.summary())\nmodel.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])\n\nmodel.fit(x_train, d1.evaluator_rating,batch_size=64,epochs=2,validation_data=(x_test,d2.evaluator_rating))"

In [61]:
"""import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score"""

'import numpy as np\nimport pandas as pd\nimport nltk\nimport re\nfrom nltk.corpus import stopwords\nfrom nltk.tokenize import sent_tokenize,word_tokenize\nfrom gensim.models import Word2Vec\nfrom keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten\nfrom keras.models import Sequential, load_model, model_from_config\nimport keras.backend as K\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import mean_squared_error\nfrom sklearn.metrics import cohen_kappa_score'

In [62]:
"""from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config

def get_model():
    model = Sequential()
    model.add(LSTM(17980, dropout=0.4, recurrent_dropout=0.4, input_shape=[1,17980], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model
lstm = get_model()"""

"from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten\nfrom keras.models import Sequential, load_model, model_from_config\n\ndef get_model():\n    model = Sequential()\n    model.add(LSTM(17980, dropout=0.4, recurrent_dropout=0.4, input_shape=[1,17980], return_sequences=True))\n    model.add(LSTM(64, recurrent_dropout=0.4))\n    model.add(Dropout(0.5))\n    model.add(Dense(1, activation='relu'))\n    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])\n    model.summary()\n    return model\nlstm = get_model()"

In [63]:
"""y_train = d1.evaluator_rating"""

'y_train = d1.evaluator_rating'

In [64]:
"""x_train.shape"""

'x_train.shape'

In [65]:
"""lstm.fit(x_train, d1.evaluator_rating , batch_size=64, epochs=1)"""

'lstm.fit(x_train, d1.evaluator_rating , batch_size=64, epochs=1)'

In [66]:
"""d1.evaluator_rating"""

'd1.evaluator_rating'

In [67]:
"""w2v_model.build_vocab(d1["essay"], progress_per=10000)
w2v_model.train(d1["essay"], total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)"""

'w2v_model.build_vocab(d1["essay"], progress_per=10000)\nw2v_model.train(d1["essay"], total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)'

In [68]:

"""model = Word2Vec(sentences=d1["essay"], vector_size = 300, window=5, min_count=1, workers=4)
model.init_sims(replace=True)
model.save("word2vec.model")
"""

'model = Word2Vec(sentences=d1["essay"], vector_size = 300, window=5, min_count=1, workers=4)\nmodel.init_sims(replace=True)\nmodel.save("word2vec.model")\n'

In [69]:
#words = model.wv.vectors

In [70]:
#words.shape

In [71]:
#model.train(d1["essay"],total_examples=model.corpus_count, epochs=1)

In [72]:
"""all_vectors = []
for index, vector in enumerate(model.wv.vectors):
    vector_object = {}
    vector_object[list(model.wv.key_to_index)[index]] = vector
    all_vectors.append(vector_object)"""

'all_vectors = []\nfor index, vector in enumerate(model.wv.vectors):\n    vector_object = {}\n    vector_object[list(model.wv.key_to_index)[index]] = vector\n    all_vectors.append(vector_object)'

In [73]:
#len(all_vectors)

In [74]:
"""import gensim
from gensim.models import Word2Vec

num_features = 500 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

print("Training Word2Vec Model...")
model = Word2Vec(sentences, workers=num_workers, vector_size=num_features, min_count = min_word_count, window = context, sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

clean_train_essays = []
    
# Generate training and testing data word vectors.
for essay_v in d1.essay:
    clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
clean_test_essays = []
for essay_v in d2.essay:
    clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
trainDataVecs = np.array(trainDataVecs)
testDataVecs = np.array(testDataVecs)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    #lstm_model = get_model()
    #lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)"""

'import gensim\nfrom gensim.models import Word2Vec\n\nnum_features = 500 \nmin_word_count = 40\nnum_workers = 4\ncontext = 10\ndownsampling = 1e-3\n\nprint("Training Word2Vec Model...")\nmodel = Word2Vec(sentences, workers=num_workers, vector_size=num_features, min_count = min_word_count, window = context, sample = downsampling)\n\nmodel.init_sims(replace=True)\nmodel.wv.save_word2vec_format(\'word2vecmodel.bin\', binary=True)\n\nclean_train_essays = []\n    \n# Generate training and testing data word vectors.\nfor essay_v in d1.essay:\n    clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))\ntrainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)\n    \nclean_test_essays = []\nfor essay_v in d2.essay:\n    clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))\ntestDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )\n    \ntrainDataVecs = np.array(trainDataVecs)\ntestDataVecs = np.array(testDataVecs)

In [75]:
#trainDataVecs.shape

In [76]:
"""for essay_v in d1.essay:
    print(essay_v)
    break"""

'for essay_v in d1.essay:\n    print(essay_v)\n    break'

In [77]:
"""def get_mean_vector(word2vec_model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in word2vec_model.key_to_index]
    if len(words) >= 1:
        return np.mean(word2vec_model[words], axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

#Vectorizing 
w2v_model = gensim.models.Word2Vec(sentences, min_count = 11, vector_size = 500, window = 10, sg=1)
"""
#words = w2v_model.wv.key_to_index
#we_dict = {word:w2v_model.wv[word] for word in words}

"""train_array=[]
test_array=[]
for line in sentences:
    vec = get_mean_vector(w2v_model.wv, line)
    if len(vec) > 0:
        train_array.append(vec)"""
        
"""for tweet in test_tweets:
    vec = get_mean_vector(w2v_model.wv, tweet)
    if len(vec) > 0:
        test_array.append(vec)"""

'for tweet in test_tweets:\n    vec = get_mean_vector(w2v_model.wv, tweet)\n    if len(vec) > 0:\n        test_array.append(vec)'

In [78]:
#len(we_dict)

In [79]:
#len(train_array)

In [80]:
#train_array

In [81]:
#BASE_MODEL = '/kaggle/input/huggingface-roberta-variants/distilroberta-base/distilroberta-base'

In [82]:
"""from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)"""

'from transformers import AutoTokenizer\n\ntokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)'

In [83]:
"""from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)"""

'from transformers import AutoTokenizer\n\ntokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)'

In [84]:
#X = d["essay"]
#Y = d["evaluator_rating"]

In [85]:
#from sklearn.model_selection import train_test_split
#x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=42)

In [86]:
"""def get_data(d1):
    tokenized = tokenizer(d1["essay"].tolist(), padding=True, return_tensors="np") 
    return {feat: tokenized[feat] for feat in tokenizer.model_input_names}
    
x = get_data(d1)
x"""

'def get_data(d1):\n    tokenized = tokenizer(d1["essay"].tolist(), padding=True, return_tensors="np") \n    return {feat: tokenized[feat] for feat in tokenizer.model_input_names}\n    \nx = get_data(d1)\nx'

In [87]:
"""#t["excerpt"] = t["excerpt"].apply(preprocess_sentence)
x_test = get_data(d2)
y_test = model.predict(x_test)
d2['target'] = y_test.logits[:,0]
d2
#t.to_csv('submission.csv', columns=['id','target'], index=False)"""

'#t["excerpt"] = t["excerpt"].apply(preprocess_sentence)\nx_test = get_data(d2)\ny_test = model.predict(x_test)\nd2[\'target\'] = y_test.logits[:,0]\nd2\n#t.to_csv(\'submission.csv\', columns=[\'id\',\'target\'], index=False)'